# MCMC to fit atmospheric parameters on measured air transmission

Use ordered dict

https://github.com/DifferentiableUniverseInitiative/jax-cosmo-paper/blob/master/notebooks/VectorizedNumPyro.ipynb

In [ ]:
from diffatmemulator.diffatmemulator import DiffAtmEmulator
from diffatmemulator.diffatmemulator import Dict_Of_sitesAltitudes,Dict_Of_sitesPressures

In [ ]:
from collections import OrderedDict

In [ ]:
from instrument.instrument import Hologram

In [ ]:
import numpy as np
import jax
import jax.numpy as jnp
from jax.scipy.special import logsumexp
jax.config.update("jax_enable_x64", True)

import numpyro
from numpyro import optim
from numpyro.diagnostics import print_summary
import numpyro.distributions as dist
from numpyro.distributions import constraints
from numpyro.infer import MCMC, HMC, NUTS, SVI, Trace_ELBO
from numpyro.infer.autoguide import AutoBNAFNormal, AutoMultivariateNormal
from numpyro.infer.reparam import NeuTraReparam
from numpyro.handlers import seed, trace, condition

import matplotlib as mpl
from matplotlib import pyplot as plt

import corner
import arviz as az
mpl.rcParams['font.size'] = 15
mpl.rcParams["figure.figsize"] = [8, 8]

In [ ]:
import os
import jax
import jax.numpy as jnp
import numpyro
import numpyro.distributions as dist

In [ ]:
def plot_params_kde(samples,hdi_probs=[0.393, 0.865, 0.989], 
                    patName=None, fname=None, pcut=None,
                   var_names=None, point_estimate="median"):
    
    if pcut is not None:
        low = pcut[0]
        up  = pcut[1] 
        #keep only data in the [low, up] percentiles ex. 0.5, 99.5
        samples={name:value[(value>np.percentile(value,low)) &  (value<np.percentile(value,up))] \
          for name, value in samples.items()}
        len_min = np.min([len(value) for name, value in samples.items()])
        len_max = np.max([len(value) for name, value in samples.items()])
        if (len_max-len_min)>0.01*len_max:
            print(f"Warning: pcut leads to min/max spls size = {len_min}/{len_max}")
        samples = {name:value[:len_min] for name, value in samples.items()}
    
    axs= az.plot_pair(
            samples,
            var_names=var_names,
            figsize=(10,10),
            kind="kde",
    #        marginal_kwargs={"plot_kwargs": {"lw": 3, "c": "b"}},
            kde_kwargs={
#                "hdi_probs": [0.68, 0.9],  # Plot 68% and 90% HDI contours
                "hdi_probs":hdi_probs,  # 1, 2 and 3 sigma contours
                "contour_kwargs":{"colors":('r', 'green', 'blue'), "linewidths":3},
                "contourf_kwargs":{"alpha":0},
            },
            point_estimate_kwargs={"lw": 3, "c": "b"},
            marginals=True, textsize=20, point_estimate=point_estimate,
        );
    
    plt.tight_layout()
    
    if patName is not None:
        patName_patch = mpatches.Patch(color='b', label=patName)
        axs[0,0].legend(handles=[patName_patch], fontsize=40, bbox_to_anchor=(1, 0.7));
    if fname is not None:
        plt.savefig(fname)
        plt.close()

# Instrument

defines the properties of the instrument measurement. The detector consists in the CCD plate of the Auxtel telescope. The Hologram class allows to retrieve the correct wavelength sampling corresponding to the measurement of a spectrum. This wavelength sampling correspond to pixel size sampling.

In [ ]:
h = Hologram(rebin=1)

In [ ]:
wls = h.get_wavelength_sample()

## Emulator

In [ ]:
obs_str = "LSST"

emul1 =  DiffAtmEmulator(obs_str=obs_str)
emul2 =  DiffAtmEmulator(obs_str=obs_str,pressure=800.)

In [ ]:
P1 = emul1.pressure
P2 = emul2.pressure

In [ ]:
transm1 = emul1.vect1d_Alltransparencies(wls,am=1,pwv=4.0,oz=400.,tau=0.1,beta=-1.2)
transm2 = emul2.vect1d_Alltransparencies(wls,am=1,pwv=4.0,oz=400.,tau=0.1,beta=-1.2)

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(6,3))
ax.plot(wls,transm1,'b',label=f"P = {P1:.1f} hPa")
ax.plot(wls,transm2,'r',label=f"P = {P2:.1f} hPa")
ax.set_xlabel("$\lambda$ (nm)")
ax.set_ylabel("transmission")
ax.set_title("mean air transparency")
ax.legend()

In [ ]:
def mean_transm(x, params,airmass):
    """ mean function of the model of transmission 
    :param x: set of wavelengths measured experimentaly (independent variable)  
    :type x: float, in unit nm
    
    :param params: dictionnary of parameters to be fitted
    :type params: float
    ...
    :raises [ErrorType]: [ErrorDescription]
    ...
    :return: return the transmission
    :rtype: float

    Assume the airmass is set to 1.0
    
    """
    pressure = params["P"]
    pwv  = params["pwv"]
    oz = params["oz"]
    tau =  params["tau"]
    beta =  params["beta"]

    airmass = airmass
    wls = x

    emul = DiffAtmEmulator(obs_str=obs_str,pressure=pressure)
    transm= emul.vect1d_Alltransparencies(wls,am=airmass,pwv=pwv,
                                          oz=oz,tau=tau,beta=beta)
    
    return transm

In [ ]:
rng_key = jax.random.PRNGKey(42)
rng_key, rng_key0, rng_key1, rng_key2 = jax.random.split(rng_key, 4)

In [ ]:
sigma_obs=0.01 # 5 mmag accuracy

In [ ]:
airmass = 1.0
par_true=OrderedDict({"P":730.0, "pwv":4.0, "oz":400, "tau": 0.05, "beta": -1.2})

In [ ]:
par_true

In [ ]:
TMes = mean_transm(wls,par_true,airmass) + sigma_obs * jax.random.normal(rng_key1,shape=wls.shape)

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(10,3))
ax.errorbar(wls,TMes,yerr=sigma_obs,fmt="o",ms=1,color="k",ecolor="r",lw=1)
ax.set_xlabel("$\lambda$ (nm)")
ax.set_ylabel("transmission")
ax.legend()
ax.grid();

In [ ]:
def model(wls,Tobs=None,
          P_min=700.,pwv_min = 0.,oz_min = 0., tau_min=0.,beta_min=-3,
          P_max=800.,pwv_max = 10.,oz_max = 550., tau_max=0.5,beta_max=0,
          airmass=1,sigma= 0.01):
    
    # priors
    pressure = numpyro.sample("P", dist.Uniform(P_min,P_max))
    pwv  = numpyro.sample("pwv", dist.Uniform(pwv_min,pwv_max))
    oz  = numpyro.sample("oz", dist.Uniform(oz_min,oz_max))
    tau= numpyro.sample("tau", dist.Uniform(tau_min,tau_max))
    beta= numpyro.sample("beta", dist.Uniform(beta_min,beta_max))
    
    params = OrderedDict({"P":pressure, "pwv":pwv, "oz":oz, "tau":tau, "beta":beta})
    mu = mean_transm(wls,params,airmass)
    
    with numpyro.plate("obs", wls.shape[0]):  # les observables sont indépendantes
        numpyro.sample('T', dist.Normal(mu, sigma), obs=Tobs)


In [ ]:
numpyro.render_model(model, model_args=(jnp.array([0.]),jnp.array([1.])),render_distributions=True)

In [ ]:
from numpyro.handlers import seed, trace, condition
# So, let's generate the data at the fiducial parameters
fiducial_model = condition(model, par_true)
# Run the conditionned model (if not conditionned the priors are executed)
trace_data = trace(seed(fiducial_model, jax.random.PRNGKey(42))).get_trace(wls, sigma=sigma_obs)

In [ ]:
#trace_data

In [ ]:
TMes_bis = trace_data['T']["value"]

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(10,3))
ax.errorbar(wls,TMes,yerr=sigma_obs,fmt='o',ms=1. ,linewidth=2, capsize=0, c='k', label="data")
ax.errorbar(wls,TMes_bis,yerr=sigma_obs, fmt='o', ms=1.,linewidth=2, capsize=0, c='r', label="data (bis)")
ax.set_xlabel("$\lambda$ (nm)")
ax.set_ylabel("transmission")
ax.legend()
ax.grid();

In [ ]:
# Run NUTS.
kernel = NUTS(model, dense_mass=True, target_accept_prob=0.9,
              init_strategy=numpyro.infer.init_to_median())
num_samples = 5_000
n_chains = 4
mcmc = MCMC(kernel, num_warmup=1_000, num_samples=num_samples,  
            num_chains=n_chains,
            chain_method='vectorized',
            progress_bar=True)
mcmc.run(rng_key, wls=wls, Tobs=TMes,sigma=sigma_obs,extra_fields=('potential_energy',))
#mcmc.run(rng_key, wls=wls, Tobs=TMes,sigma=sigma_obs)
mcmc.print_summary()
samples_nuts = mcmc.get_samples()

In [ ]:
az.ess(samples_nuts, relative=True)  # efficacité relative

In [ ]:
data = az.from_numpyro(mcmc)
az.plot_trace(data, compact=False);
plt.tight_layout()

In [ ]:
plot_params_kde(samples_nuts, pcut=[0.001,99.999], var_names=['P', 'pwv','oz', 'tau','beta'])

In [ ]:
wls_val = np.linspace(300.,1000,100)

In [ ]:
Ttrue_val = mean_transm(wls_val,par_true,airmass)

In [ ]:
func = jax.vmap(lambda x: mean_transm(wls_val,x,airmass))

In [ ]:
Tall_val= func(samples_nuts) 

In [ ]:
Tmean_val = jnp.mean(Tall_val,axis=0)

In [ ]:
std_T_val = jnp.std(Tall_val,axis=0)

In [ ]:
fig,ax =plt.subplots(1,1,figsize=(10,4))
ax.errorbar(wls,TMes,yerr=sigma_obs,fmt='o', ms=1,linewidth=1, capsize=0, c='k', label="data")
ax.plot(wls_val,Ttrue_val,c='k',label="true")

ax.fill_between(wls_val, Tmean_val-2*std_T_val, Tmean_val+2*std_T_val, 
                    color="lightblue",label=r"$2-\sigma$")
ax.fill_between(wls_val, Tmean_val-std_T_val, Tmean_val+std_T_val, 
                    color="lightgray",label=r"$1-\sigma$")
# plot mean prediction
ax.plot(wls_val, Tmean_val, "blue", ls="--", lw=2.0, label="mean")


ax.set_xlabel("$\lambda$ (nm)")
ax.set_ylabel("transmission")
ax.legend()
ax.grid();

In [ ]:
samples_nuts

In [ ]:
reordered_samples = OrderedDict()

In [ ]:
par_names = list(par_true.keys())
par_names

In [ ]:
for key in par_true.keys():
    reordered_samples[key] = samples_nuts[key]
    

In [ ]:
reordered_samples

In [ ]:
nparams = len(reordered_samples)
nparams

In [ ]:
data = reordered_samples

In [ ]:
import arviz.labels as azl

labeller = azl.MapLabeller(var_name_map={"P": r"$P$", 
                                         "pwv":r"$H_2O$",
                                         "oz":r"$O_3$",
                                         "tau":r"$\tau$",
                                         "beta":r"$\beta$" 
                                        })


In [ ]:
ax=az.plot_pair(
        data,
        kind="kde",
        labeller=labeller,
        marginal_kwargs={"plot_kwargs": {"lw":3, "c":"blue", "ls":"-"}},
        kde_kwargs={
            "hdi_probs": [0.3, 0.68, 0.9],  # Plot 30%, 68% and 90% HDI contours
            "contour_kwargs":{"colors":None, "cmap":"Blues", "linewidths":3,
                              "linestyles":"-"},
            "contourf_kwargs":{"alpha":0.5},
        },
        point_estimate_kwargs={"lw": 3, "c": "b"},
        marginals=True, textsize=50, point_estimate='median',
    );

# plot true parameter point
for idy in range(nparams):
    for idx in range(idy):
        label_x = par_names[idx]
        label_y = par_names[idy]
        ax[idy,idx].scatter(par_true[label_x],par_true[label_y],c="r",s=150,zorder=10)
        

for idx,name in enumerate(par_names):
    ax[idx,idx].axvline(par_true[name],c='r',lw=3)
    